In [153]:
#%pip install scikit-learn
#%pip install nltk
#%pip install utils
#%pip install emoji
#%pip install unidecode

                                              0.0/235.9 kB ? eta -:--:--
     ------------------                     112.6/235.9 kB 3.3 MB/s eta 0:00:01
     -------------------------------------- 235.9/235.9 kB 2.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [183]:
import pandas as pd
import re
#from sklarn.feature_extraction.text import CountVectorizer
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
import numpy as np
import sys
sys.path.append('../')
#from utils import remove_similar_rows, find_lines_with_player, remove_similar_rows_per_player, remove_stopwords_from_text
#from utils import find_lines_with_player, map_emoji_to_description, remove_similar_rows_per_player ##del_patterns, 


#INFO included in utils - but utils doesn't work for me here
import emoji
def map_emoji_to_description(emoji_text, language): 
    emoji_description = emoji.demojize(emoji_text, language=language)
    return emoji_description

def translate_emojis(text, language):
    return re.sub(r'[\U0001F000-\U0001F999]', lambda match: map_emoji_to_description(match.group(), language=language), text)

def remove_stopwords_from_text(text, stopwords_list_per_language):
    return remove_stopwords(text, stopwords=stopwords_list_per_language)

from difflib import SequenceMatcher
def remove_similar_rows_per_player(df, playerlist, threshold=0.9):
    '''The procedure of deleting similiar articles needs to be done by each player because if an article writes about 
    # e.g. two players we want to keep it for both of the players'''

    # define empty df which will be returned in the end
    df_complete = pd.DataFrame()

    for player in playerlist:
        
        # create the df for the player
        df_player = df[df["player"] == player]
        df_player = df_player.reset_index(drop=True)
        column_as_df = pd.DataFrame(df_player['data'])


        
        # Compute similarity scores for each pair of rows
        similarity_scores = {}
        for i, row in column_as_df.iterrows():
            for j, other_row in column_as_df.iterrows():
                if i >= j:
                    continue
                score = SequenceMatcher(None, row, other_row).ratio()
                if score >= threshold:
                    similarity_scores[(i, j)] = score
        
        # Identify rows to remove
        rows_to_remove = []
        for (i, j), score in similarity_scores.items():
            if i not in rows_to_remove and j not in rows_to_remove:
                rows_to_remove.append(j if df_player.index[i] < df_player.index[j] else i)
        
        # Remove rows and concatenate df
        df_player = df_player.drop(rows_to_remove)
        df_complete = pd.concat([df_complete, df_player], axis=0)

        #return modified DataFrame
    return df_complete

from unidecode import unidecode
def remove_accents(text):
    return unidecode(text)

'''def del_patterns(df_line, pattern):
    lines = df_line.split('\\n')
    new_string = ''
    for line in lines:
        deleting = False
        for word in pattern:
            if deleting:
                break
            elif word in line:
                deleting = True
            else:
                deleting = False
        if not deleting:
            new_string += line + '\n'
    return new_string.strip()'''
def del_patterns(df_line, pattern):
    '''
    Function which takes an input and deletes defined text pattern 
    '''
    # split up the records in lines
    lines = df_line.split('\\n')
    '''
    lines = df_line.split('\n')  # Split at newline characters first

    split_lines = []
    for line in lines:
        split_lines.extend(line.split('\\t'))  # Split each line at tab characters

    '''

    if len(lines)>1:

        # create an empty string
        new_string = ''

        # iterating over the lines 
        for line in lines:
        
        # set deleting to False first
            deleting = False
            # check if a pattern word is included in the line and set deleting to True if so 
            for word in pattern:
                if deleting == True:
                    break
                elif word in line:
                    deleting = True
                else:
                    deleting = False
            
            # if the setence should not be delete it add it to the string  
            if deleting == False:
                new_string = new_string + line 

    else:
        new_string = df_line
    # return the string 
    return new_string

# Load Data

In [3]:
url = 'https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/data_files/all_data_v3.csv'
df = pd.read_csv(url)

In [4]:
# Filter out the Spanish data and reindex
df_en = df[df["language"] == "en"]

#Reset index
df_en = df_en.reset_index(drop=True)
df_en

,data,player,language,publishedAt
0,{'content': 'Football\nFlorian Wirtz\'s goal f...,Exequiel Palacios,en,2023-02-16T23:56:00Z
1,{'content': '[1/4]\xa0Soccer Football - Europa...,Exequiel Palacios,en,2023-02-23T20:50:50Z
2,"{'content': ""By Will Pickworth For Mailonline\...",Exequiel Palacios,en,2023-02-23T20:53:59Z
3,{'content': '\nBUENOS AIRES (AP) — World Cup w...,Exequiel Palacios,en,2023-03-03T16:40:46Z
4,{'content': 'Sign In\nSign In\nThe Star Editio...,Exequiel Palacios,en,2023-03-03T16:42:19Z
...,...,...,...,...
297,{'content': 'Please refresh the page or naviga...,Moussa Diaby,en,2023-05-04T06:28:32Z
298,"{'content': ""By Oliver Salt For Mailonline\nPu...",Mykhaylo Mudryk,en,2023-05-05T14:58:51Z
299,{'content': 'We use cookies and other tracking...,Piero Hincapié,en,2023-04-27T04:57:02Z
300,{'content': 'Man City’s Alex Robertson makes d...,Piero Hincapié,en,2023-03-24T15:24:08Z


In [5]:
print(df_en.loc[5, 'data'])

{'content': "World Cup winners Argentina will celebrate with their fans at two home games in March against Panama and Curacao in their first matches since their triumph in Qatar.\nArgentina coach Lionel Scaloni announced a 35-man squad on Friday which included Lionel Messi, Angel Di Maria and the other members of Argentina's winning squad in Qatar.\nManchester United forward Alejandro Garnacho is rewarded for impressing at Old Trafford with his first-ever callup. The 18-year-old, in his breakthrough season with Erik ten Hag's side, has contributed five goals and six assists in 30 matches this campaign.\nIn their first games since beating France in the World Cup final, Argentina will play Panama on March 23 at the Monumental Stadium in Buenos Aires. Five days later, they will host Curacao in another friendly at the northern city of Santiago del Estero.\nScaloni, 44, earlier this week extended his contract until the end of the 2026 World Cup.\nSquad:\nGoalkeepers: Emiliano Martinez (Asto

# Remove similiar rows

In [6]:
df_en = remove_similar_rows_per_player(df_en, df_en['player'].unique())
df_en

,data,player,language,publishedAt
0,{'content': 'Football\nFlorian Wirtz\'s goal f...,Exequiel Palacios,en,2023-02-16T23:56:00Z
1,{'content': '[1/4]\xa0Soccer Football - Europa...,Exequiel Palacios,en,2023-02-23T20:50:50Z
2,"{'content': ""By Will Pickworth For Mailonline\...",Exequiel Palacios,en,2023-02-23T20:53:59Z
3,{'content': '\nBUENOS AIRES (AP) — World Cup w...,Exequiel Palacios,en,2023-03-03T16:40:46Z
4,{'content': 'Sign In\nSign In\nThe Star Editio...,Exequiel Palacios,en,2023-03-03T16:42:19Z
...,...,...,...,...
19,"{'content': ""By Australian Associated Press\nP...",Piero Hincapie,en,2023-03-28T11:24:40Z
20,{'content': 'Julien Laurens explains why there...,Piero Hincapie,en,2023-03-29T11:35:08Z
0,{'content': 'We use cookies and other tracking...,Piero Hincapié,en,2023-04-27T04:57:02Z
1,{'content': 'Man City’s Alex Robertson makes d...,Piero Hincapié,en,2023-03-24T15:24:08Z


# Transform data into lower case

In [7]:
data_lower_en = df_en.copy()

data_lower_en['data'] = data_lower_en['data'].str.lower()
data_lower_en['player'] = data_lower_en['player'].str.lower()
data_lower_en.head()

,data,player,language,publishedAt
0,{'content': 'football\nflorian wirtz\'s goal f...,exequiel palacios,en,2023-02-16T23:56:00Z
1,{'content': '[1/4]\xa0soccer football - europa...,exequiel palacios,en,2023-02-23T20:50:50Z
2,"{'content': ""by will pickworth for mailonline\...",exequiel palacios,en,2023-02-23T20:53:59Z
3,{'content': '\nbuenos aires (ap) — world cup w...,exequiel palacios,en,2023-03-03T16:40:46Z
4,{'content': 'sign in\nsign in\nthe star editio...,exequiel palacios,en,2023-03-03T16:42:19Z


# Remove patterns

In [188]:
print(data_lower_en.iloc[4].values)

["{'content': 'sign in\\nsign in\\nthe star edition\\nchange location\\nthis copy is for your personal non-commercial use only. to order\\n    presentation-ready copies of toronto star content for distribution\\n    to colleagues, clients or customers, or inquire about\\n    permissions/licensing, please go to: www.torontostarreprints.com\\nbuenos aires (ap) — world cup winner argentina will celebrate with its fans at two home games in march against panama and curacao.\\nargentina coach lionel scaloni on friday announced a 35-man squad which included lionel messi, ángel di maria and the other members of argentina’s winning squad in qatar.\\nin its first games since beating france in the world cup final, argentina will play panama on march 23 at the monumental stadium in buenos aires. five days later it will host curacao in another friendly at the northern city of santiago del estero.\\nmanchester united forward alejandro garnacho is rewarded for impressing at manchester united with his

In [189]:
data_wo_pattern_en = data_lower_en.copy()

# delete content patterns
data_wo_pattern_en['data'] = data_wo_pattern_en['data'].apply(lambda x: re.sub(r"^{\'content\': \'", "", str(x)))
data_wo_pattern_en['data'] = data_wo_pattern_en['data'].apply(lambda x: re.sub(r"{'content':", "", str(x)))

#remove patterns
patternlist_en = [
    "copyright",
    "photo",
    "all rights reserved",
    'published by',
    'published',    
    'updated',
    'last updated',
    'from the section',
    'filed under:',
    'when you purchase through links on our site, we may earn an affiliate commission',
    'how it works',
    'to the independent?',
    'if you would prefer:',
    'want an ad-free experience?\n',
    'refresh the page or navigate to another page on the site to be automatically logged inplease refresh your browser to be logged in',
    'your bookmarks in your independent premium section, under my profile',
    'to bookmark your favourite articles and stories to read or reference later? start your independent premium subscription today.please refresh the page or navigate to another page on the site to be automatically logged inplease refresh your browser to be logged inlog in',
    'use cookies and other tracking technologies to improve your browsing experience on our site, show personalized content and targeted ads, analyze site traffic, and understand where our audiences come from. to learn more or opt-out, read our cookie policy. please also read our privacy notice and terms of use, which became effective december 20, 2019.',
    'by choosing i accept, you consent to our use of cookies and other tracking technologies.'
]

#ACTION: pattern I still can't make working:
    '''sign in\\nsign in\\nthe star edition\\nchange location\\nthis copy is for your personal non-commercial use only. to order\\n    presentation-ready copies of toronto star content for distribution\\n    to colleagues, clients or customers, or inquire about\\n    permissions/licensing, please go to: www.torontostarreprints.com',
    'sign insign inthe star editionchange locationthis copy is for your personal non-commercial use only. to order',
    'sign insign in',
    'presentation-ready copies of toronto star content for distribution    to colleagues, clients or customers, or inquire about',
    'permissions/licensing, please go to: www.torontostarreprints'''

In [190]:
data_wo_pattern_en['data'] = data_wo_pattern_en['data'].apply(lambda x: del_patterns(str(x), patternlist_en))                                                      

In [191]:
#ACTION
#check whether it worked
data_affected_row = data_wo_pattern_en.copy()
filtered_rows = data_affected_row[data_affected_row['data'].str.contains('sign in', case=False)]

# Display the filtered rows
df_filtered = pd.DataFrame(filtered_rows['data'])
df_filtered
#print(df_filtered.iloc[0].values)

,data
4,sign insign inthe star editionchange locationt...
20,check out the transfer rumours of all the play...
34,sign insign inthe star editionchange locationt...
58,the january transfer window may be closed arou...
75,sign insign inthe star editionchange locationt...
1,sign insign inthe star editionchange locationt...
5,sign insign inthe star editionchange locationt...
9,"""sign insign inthe star editionchange locatio..."
20,sign insign inthe star editionchange locationt...
31,sign insign inthe star editionchange locationt...


# Translate emojis into words

In [167]:
data_wo_emojis = data_wo_pattern_en.copy()

data_wo_emojis['data'] = data_wo_emojis['data'].apply(lambda x: translate_emojis(str(x), language='en'))

# Remove noise

In [214]:
data_rm_en = data_wo_emojis.copy()

# strip_numeric -> removing digits (https://gensimr.news-r.org/reference/strip_numeric.html)
data_rm_en['data'] = data_rm_en['data'].apply(strip_numeric)

#strip links
data_rm_en['data'] = data_rm_en['data'].apply(lambda x: re.sub(r'http\S+', '', str(x)))

# strip_punctutation -> removing punctations (https://gensimr.news-r.org/reference/strip_punctuation.html)
data_rm_en['data'] = data_rm_en['data'].apply(strip_punctuation)

# strip multiple whitespaces also \n -> (https://radimrehurek.com/gensim/parsing/preprocessing.html#gensim.parsing.preprocessing.strip_multiple_whitespaces)
data_rm_en['data'] = data_rm_en['data'].apply(strip_multiple_whitespaces)

#strip spanish accents
data_rm_en['data'] = data_rm_en['data'].apply(lambda x: remove_accents(str(x)))

#strip images
#ACTION: I wouldn't do it for englisch because there is only one datapoint affected

#strip_short deletes words smaller 3
data_rm_en['data'] = data_rm_en['data'].apply(strip_short)
 

data_rm_en

,data,player,language,publishedAt
0,footballflorian wirtz goal for bayer leverkuse...,exequiel palacios,en,2023-02-16T23:56:00Z
1,xasoccer football europa league play off secon...,exequiel palacios,en,2023-02-23T20:50:50Z
2,will pickworth for mailonlineview commentsbaye...,exequiel palacios,en,2023-02-23T20:53:59Z
3,buenos aires world cup winner argentina will c...,exequiel palacios,en,2023-03-03T16:40:46Z
4,sign insign inthe star editionchange locationt...,exequiel palacios,en,2023-03-03T16:42:19Z
...,...,...,...,...
19,australian associated pressview commentsthe so...,piero hincapie,en,2023-03-28T11:24:40Z
20,julien laurens explains why there was only one...,piero hincapie,en,2023-03-29T11:35:08Z
0,one for the futurechelsea were rumored have be...,piero hincapié,en,2023-04-27T04:57:02Z
1,man city's alex robertson makes debut aiden o'...,piero hincapié,en,2023-03-24T15:24:08Z


# Remove Stopwords

In [ ]:
# Words to keep
words_to_keep = {'don', 'didn', 'doesn', 'shouldn', 'couldn', 'wouldn', 'never', 'isn', 'cannot', 'except', 'no', 'neither', 'nor', 'cant', 'top', 'least'}

# Create modified stopwords list
modified_en_stopwords = STOPWORDS - words_to_keep

print(modified_en_stopwords)

frozenset({'more', 'my', 'would', 'did', 'somewhere', 'made', 'than', 'even', 'their', 'often', 'perhaps', 'alone', 'when', 'thin', 'elsewhere', 'latter', 'another', 'she', 'regarding', 'fill', 'thick', 're', 'quite', 'ten', 'using', 'make', 'too', 'fifteen', 'eleven', 'sincere', 'have', 'formerly', 'may', 'become', 'since', 'five', 'empty', 'bottom', 'his', 'somehow', 'etc', 'beyond', 'almost', 'thereby', 'meanwhile', 'a', 'into', 'see', 'sometimes', 'whither', 'behind', 'one', 'wherein', 'are', 'always', 'same', 'without', 'computer', 'latterly', 'along', 'where', 'con', 'off', 'at', 'serious', 'ltd', 'themselves', 'toward', 'across', 'through', 'forty', 'herself', 'most', 'kg', 'nevertheless', 'though', 'everyone', 'moreover', 'each', 'will', 'last', 'de', 'however', 'amoungst', 'over', 'due', 'name', 'with', 'been', 'that', 'everywhere', 'enough', 'whole', 'twelve', 'amount', 'might', 'very', 'together', 'inc', 'describe', 'someone', 'under', 'by', 'had', 'call', 'whatever', 'give'

In [ ]:
data_rm_stop_en = data_rm_en.copy()

# Apply the remove_stopwords function to the 'text' column using the apply method
data_rm_stop_en['data'] = data_rm_stop_en['data'].apply(lambda x: remove_stopwords_from_text(x, modified_en_stopwords))

# Reset index

In [ ]:
data_en_pp = data_rm_stop_en.copy()

data_en_pp.reset_index(drop=True, inplace=True)

# Tokenization

In [ ]:
#ACTION

# Store data in csv

In [ ]:
#ACTION
'''
import os
# Define the folder path
folder_path = "./data_files"

# Define the file path
file_path = os.path.join(folder_path, "en_data_preprocessed.csv")

# Save the DataFrame as a CSV file
data_en_pp.to_csv(file_path, index=False)

print("Data saved successfully.")'''

# Code to check whether preprocessing worked

In [ ]:
'''Control for patterns
#check whether it worked
data_affected_row = data_wo_pattern_en.copy()
filtered_rows = data_affected_row[data_affected_row['data'].str.contains('by choosing', case=False)]

# Display the filtered rows
df_filtered = pd.DataFrame(filtered_rows['data'])
df_filtered
#print(df_filtered.iloc[0].values)
'''

In [ ]:
'''Control for emojis

# Unicode ranges for emojis
emoji_ranges = [
    (0x1F600, 0x1F64F),  # Emoticons
    (0x1F300, 0x1F5FF),  # Miscellaneous symbols and pictographs
    (0x1F680, 0x1F6FF),  # Transport and map symbols
    (0x2600, 0x26FF),    # Miscellaneous symbols
    (0x2700, 0x27BF),    # Dingbats
    (0xFE00, 0xFE0F),    # Variation Selectors
    (0x1F900, 0x1F9FF),  # Supplemental Symbols and Pictographs
    (0x1F1E6, 0x1F1FF)   # Flags
]

# Function to check if a character is an emoji
def is_emoji(character):
    if emoji.demojize(character) != character:
        return True
    return False

# Assuming your DataFrame is named 'df'
articles_with_untranslated_emojis = 0

# Iterate over the rows of the DataFrame
for index, row in data_wo_emojis.iterrows():
    # Counter for untranslated emojis in the current row
    untranslated_emoji_count = 0

    # Iterate over the characters in the row
    for char in str(row['data']):
        if is_emoji(char) and emoji.demojize(char) == char:
            untranslated_emoji_count += 1

    # If there is at least one untranslated emoji in the current row, increment the count of rows with untranslated emojis
    if untranslated_emoji_count > 0:
        articles_with_untranslated_emojis += 1
        print("Untranslated emojis found in row", index + 1, ":", untranslated_emoji_count)

print("Total number of rows with untranslated emojis:", articles_with_untranslated_emojis)
'''